# Household Energy Consumption Patterns

This notebook focuses on energy consumption by households across EU countries, leveraging SDMX datasets.

## What the data reveals:
- **Households are major energy consumers**: The residential sector consistently accounts for a large share of final energy consumption.
- **Country-level differences**: There are notable cross-country differences in household energy use per capita and per household, reflecting climatic, behavioral, and policy differences.
- **Recent trends**: The data allows us to observe the impact of recent events (e.g. energy crises or efficiency programs) on household consumption patterns over time.

This analysis provides a basis for understanding how policy measures or external shocks affect energy usage at the household level.

Try to add a relationship to temperature extremes

In [1]:
from utils import utils
import json
import plotly.express as px


with open('utils/sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)


In [2]:
average_household_constraints = {
    'freq': 'A',
    'unit': 'AVG',
    'geo': sub_codelists['geo_eu_countries']
}

nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': 'FC_OTH_HH_E',
    'siec': 'TOTAL',
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_eu_countries'],
    }

In [3]:
average_household = utils.get_dataset_with_selection('ilc_lvph01', average_household_constraints)

demo_pjan = utils.get_dataset_with_selection('demo_pjan', demo_pjan_constraints)
demo_pjan.data.TIME_PERIOD  = demo_pjan.data.TIME_PERIOD  - 1

nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints)


data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_lvph01/A.AVG.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/ilc_lvph01/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/demo_pjan/A.NR.TOTAL.T.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/demo_pjan/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_bal_s/A.FC_OTH_HH_E.TOTAL.GWH.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_bal_s/l

In [8]:

demo_pjan_df = demo_pjan.data.drop(columns=['OBS_FLAG', 'freq', 'unit', 'age', 'sex'])
demo_pjan_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

average_household_df = average_household.data.drop(columns=['OBS_FLAG', 'freq', 'unit'])
average_household_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

number_of_households_df = demo_pjan_df.div(average_household_df)
display(number_of_households_df)


OBS_VALUE
geo TIME_PERIOD           
AT  2003               2.4
    2004               2.4
    2005               2.3
    2006               2.3
    2007               2.3
...                    ...
SK  2019               2.9
    2020               2.9
    2021               2.9
    2022               3.1
    2023               3.1

[544 rows x 1 columns]

In [ ]:
nrg_bal_df = nrg_bal.data.set_index(['geo', 'TIME_PERIOD'])

display(nrg_bal.data)

In [6]:
consumption_per_household = nrg_bal_df.div(number_of_households_df)

In [ ]:
consumption_per_household.reset_index(inplace=True)
consumption_per_household['OBS_VALUE'] = consumption_per_household['OBS_VALUE'] * 1000
consumption_per_household.rename(columns={'OBS_VALUE': 'consumption_per_household_MWh'}, inplace=True)

fig = px.line(consumption_per_household, x='TIME_PERIOD', y='consumption_per_household_MWh', color='geo')
fig.show()
